In [32]:
import torch.nn as nn
from datasets import load_multitask_data
import bert
from config import PretrainedConfig, BertConfig
import torch
from datasets import SentencePairDataset
from tokenizer import BertTokenizer
from base_bert import BertPreTrainedModel
from datasets import load_multitask_data, SentenceClassificationDataset, SentencePairDataset
from config import PretrainedConfig, BertConfig
import torch
from base_bert import BertPreTrainedModel
from bert import BertLayer, BertModel, BertPreTrainedModel

In [42]:
def create_attention_mask(tensor):
    # Get the first occurrence of 102 for each row in the tensor
    first_occurrence_indices = find_first_occurrence(tensor)

    # Create an attention mask tensor filled with ones
    attention_mask = torch.ones_like(tensor)

    # Set the values before the first occurrence of 102 to zeros
    for i, idx in enumerate(first_occurrence_indices):
        if idx is not None:
            attention_mask[i, :idx + 1] = 0
        else:
            attention_mask[i] = 0

    return attention_mask

In [3]:
import time, random, numpy as np, argparse, sys, re, os
from types import SimpleNamespace

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from bert import BertModel
from optimizer import AdamW
from tqdm import tqdm

from datasets import SentenceClassificationDataset, SentencePairDataset, \
    load_multitask_data, load_multitask_test_data

from evaluation import model_eval_sst, test_model_multitask


TQDM_DISABLE=True

# fix the random seed
def seed_everything(seed=11711):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


BERT_HIDDEN_SIZE = 768
N_SENTIMENT_CLASSES = 5


class MultitaskBERT(nn.Module):
    '''
    This module should use BERT for 3 tasks:

    - Sentiment classification (predict_sentiment)
    - Paraphrase detection (predict_paraphrase)
    - Semantic Textual Similarity (predict_similarity)
    '''
    def __init__(self, config):
        super(MultitaskBERT, self).__init__()
        # You will want to add layers here to perform the downstream tasks.
        # Pretrain mode does not require updating bert paramters.
        #self.num_labels = config.num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')#, local_files_only=config.local_files_only)
        for param in self.bert.parameters():
            if config.option == 'pretrain':
                param.requires_grad = False
            elif config.option == 'finetune':
                param.requires_grad = True
        ### TODO
        ## sst - sentiment, para, sts - semantic text similarity    
        self.drop = torch.nn.Dropout(p=0.3)
        self.sst_classifier = torch.nn.Linear(self.bert.config.hidden_size, self.num_labels_sst)
        self.para_classifier = torch.nn.Linear(self.bert.config.hidden_size, self.num_labels_para)
        self.sts_classifier = torch.nn.Linear(self.bert.config.hidden_size, self.num_labels_sts)



    def forward(self, input_ids, attention_mask):
        'Takes a batch of sentences and produces embeddings for them.'
        # The final BERT embedding is the hidden state of [CLS] token (the first token)
        # Here, you can start by just returning the embeddings straight from BERT.
        # When thinking of improvements, you can later try modifying this
        # (e.g., by adding other layers).
        bert_out = self.bert(input_ids, attention_mask) 
        dropped = self.drop(bert_out['pooler_output'])
        out = self.linear(dropped)
        sentence_embeddings = out.last_hidden_state[:, 0, :]
        return sentence_embeddings


    def predict_sentiment(self, input_ids, attention_mask):
        '''Given a batch of sentences, outputs logits for classifying sentiment.
        There are 5 sentiment classes:
        (0 - negative, 1- somewhat negative, 2- neutral, 3- somewhat positive, 4- positive)
        Thus, your output should contain 5 logits for each sentence.
        '''
        ### TODO
        raise NotImplementedError


    def predict_paraphrase(self,
                           input_ids_1, attention_mask_1,
                           input_ids_2, attention_mask_2):
        '''Given a batch of pairs of sentences, outputs a single logit for predicting whether they are paraphrases.
        Note that your output should be unnormalized (a logit); it will be passed to the sigmoid function
        during evaluation, and handled as a logit by the appropriate loss function.
        '''
        ### TODO
        raise NotImplementedError


    def predict_similarity(self,
                           input_ids_1, attention_mask_1,
                           input_ids_2, attention_mask_2):
        '''Given a batch of pairs of sentences, outputs a single logit corresponding to how similar they are.
        Note that your output should be unnormalized (a logit); it will be passed to the sigmoid function
        during evaluation, and handled as a logit by the appropriate loss function.
        '''
        ### TODO
        raise NotImplementedError




def save_model(model, optimizer, args, config, filepath):
    save_info = {
        'model': model.state_dict(),
        'optim': optimizer.state_dict(),
        'args': args,
        'model_config': config,
        'system_rng': random.getstate(),
        'numpy_rng': np.random.get_state(),
        'torch_rng': torch.random.get_rng_state(),
    }

    torch.save(save_info, filepath)
    print(f"save the model to {filepath}")


## Currently only trains on sst dataset
def train_multitask(args):
    device = torch.device('cuda') if args.use_gpu else torch.device('cpu')
    # Load data
    # Create the data and its corresponding datasets and dataloader
    sst_train_data, num_labels,para_train_data, sts_train_data = load_multitask_data(args.sst_train,args.para_train,args.sts_train, split ='train')
    sst_dev_data, num_labels,para_dev_data, sts_dev_data = load_multitask_data(args.sst_dev,args.para_dev,args.sts_dev, split ='train')

    sst_train_data = SentenceClassificationDataset(sst_train_data, args)
    sst_dev_data = SentenceClassificationDataset(sst_dev_data, args)

    sst_train_dataloader = DataLoader(sst_train_data, shuffle=True, batch_size=args.batch_size,
                                      collate_fn=sst_train_data.collate_fn)
    sst_dev_dataloader = DataLoader(sst_dev_data, shuffle=False, batch_size=args.batch_size,
                                    collate_fn=sst_dev_data.collate_fn)

    # Init model
    config = {'hidden_dropout_prob': args.hidden_dropout_prob,
              'num_labels': num_labels,
              'hidden_size': 768,
              'data_dir': '.',
              'option': args.option,
              'local_files_only': args.local_files_only}

    config = SimpleNamespace(**config)

    model = MultitaskBERT(config)
    model = model.to(device)

    lr = args.lr
    optimizer = AdamW(model.parameters(), lr=lr)
    best_dev_acc = 0

    # Run for the specified number of epochs
    for epoch in range(args.epochs):
        model.train()
        train_loss = 0
        num_batches = 0
        for batch in tqdm(sst_train_dataloader, desc=f'train-{epoch}', disable=TQDM_DISABLE):
            b_ids, b_mask, b_labels = (batch['token_ids'],
                                       batch['attention_mask'], batch['labels'])

            b_ids = b_ids.to(device)
            b_mask = b_mask.to(device)
            b_labels = b_labels.to(device)

            optimizer.zero_grad()
            logits = model.predict_sentiment(b_ids, b_mask)
            loss = F.cross_entropy(logits, b_labels.view(-1), reduction='sum') / args.batch_size

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            num_batches += 1

        train_loss = train_loss / (num_batches)

        train_acc, train_f1, *_ = model_eval_sst(sst_train_dataloader, model, device)
        dev_acc, dev_f1, *_ = model_eval_sst(sst_dev_dataloader, model, device)

        if dev_acc > best_dev_acc:
            best_dev_acc = dev_acc
            save_model(model, optimizer, args, config, args.filepath)

        print(f"Epoch {epoch}: train loss :: {train_loss :.3f}, train acc :: {train_acc :.3f}, dev acc :: {dev_acc :.3f}")



def test_model(args):
    with torch.no_grad():
        device = torch.device('cuda') if args.use_gpu else torch.device('cpu')
        saved = torch.load(args.filepath)
        config = saved['model_config']

        model = MultitaskBERT(config)
        model.load_state_dict(saved['model'])
        model = model.to(device)
        print(f"Loaded model to test from {args.filepath}")

        test_model_multitask(args, model, device)


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--sst_train", type=str, default="data/ids-sst-train.csv")
    parser.add_argument("--sst_dev", type=str, default="data/ids-sst-dev.csv")
    parser.add_argument("--sst_test", type=str, default="data/ids-sst-test-student.csv")

    parser.add_argument("--para_train", type=str, default="data/quora-train.csv")
    parser.add_argument("--para_dev", type=str, default="data/quora-dev.csv")
    parser.add_argument("--para_test", type=str, default="data/quora-test-student.csv")

    parser.add_argument("--sts_train", type=str, default="data/sts-train.csv")
    parser.add_argument("--sts_dev", type=str, default="data/sts-dev.csv")
    parser.add_argument("--sts_test", type=str, default="data/sts-test-student.csv")

    parser.add_argument("--seed", type=int, default=11711)
    parser.add_argument("--epochs", type=int, default=10)
    parser.add_argument("--option", type=str,
                        help='pretrain: the BERT parameters are frozen; finetune: BERT parameters are updated',
                        choices=('pretrain', 'finetune'), default="pretrain")
    parser.add_argument("--use_gpu", action='store_true')

    parser.add_argument("--sst_dev_out", type=str, default="predictions/sst-dev-output.csv")
    parser.add_argument("--sst_test_out", type=str, default="predictions/sst-test-output.csv")

    parser.add_argument("--para_dev_out", type=str, default="predictions/para-dev-output.csv")
    parser.add_argument("--para_test_out", type=str, default="predictions/para-test-output.csv")

    parser.add_argument("--sts_dev_out", type=str, default="predictions/sts-dev-output.csv")
    parser.add_argument("--sts_test_out", type=str, default="predictions/sts-test-output.csv")

    # hyper parameters
    parser.add_argument("--batch_size", help='sst: 64 can fit a 12GB GPU', type=int, default=64)
    parser.add_argument("--hidden_dropout_prob", type=float, default=0.3)
    parser.add_argument("--lr", type=float, help="learning rate, default lr for 'pretrain': 1e-3, 'finetune': 1e-5",
                        default=1e-3)
    parser.add_argument("--local_files_only", action='store_true')

    args = parser.parse_args()
    return args

if __name__ == "__main__":
    args = get_args()
    args.filepath = f'{args.option}-{args.epochs}-{args.lr}-multitask.pt' # save path
    seed_everything(args.seed)  # fix the seed for reproducibility
    train_multitask(args)
    test_model(args)


usage: ipykernel_launcher.py [-h] [--sst_train SST_TRAIN] [--sst_dev SST_DEV]
                             [--sst_test SST_TEST] [--para_train PARA_TRAIN]
                             [--para_dev PARA_DEV] [--para_test PARA_TEST]
                             [--sts_train STS_TRAIN] [--sts_dev STS_DEV]
                             [--sts_test STS_TEST] [--seed SEED]
                             [--epochs EPOCHS] [--option {pretrain,finetune}]
                             [--use_gpu] [--sst_dev_out SST_DEV_OUT]
                             [--sst_test_out SST_TEST_OUT]
                             [--para_dev_out PARA_DEV_OUT]
                             [--para_test_out PARA_TEST_OUT]
                             [--sts_dev_out STS_DEV_OUT]
                             [--sts_test_out STS_TEST_OUT]
                             [--batch_size BATCH_SIZE]
                             [--hidden_dropout_prob HIDDEN_DROPOUT_PROB]
                             [--lr LR] [--local_files_only]
ip

SystemExit: 2

c:\Users\kacpe\anaconda3\envs\dnlp\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [36]:
# Get initialization parameters to validate the methods we create
class BertConfig(PretrainedConfig):
  model_type = "bert"

  def __init__(
    self,
    vocab_size=30522,
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    max_position_embeddings=512,
    type_vocab_size=2,
    initializer_range=0.02,
    layer_norm_eps=1e-12,
    pad_token_id=0,
    gradient_checkpointing=False,
    position_embedding_type="absolute",
    use_cache=True,
    name_or_path = "checkpoint",
    **kwargs
  ):
    super().__init__(pad_token_id=pad_token_id, **kwargs)

    self.vocab_size = vocab_size
    self.hidden_size = hidden_size
    self.num_hidden_layers = num_hidden_layers
    self.num_attention_heads = num_attention_heads
    self.hidden_act = hidden_act
    self.intermediate_size = intermediate_size
    self.hidden_dropout_prob = hidden_dropout_prob
    self.attention_probs_dropout_prob = attention_probs_dropout_prob
    self.max_position_embeddings = max_position_embeddings
    self.type_vocab_size = type_vocab_size
    self.initializer_range = initializer_range
    self.layer_norm_eps = layer_norm_eps
    self.gradient_checkpointing = gradient_checkpointing
    self.position_embedding_type = position_embedding_type
    self.use_cache = use_cache
    self.name_or_path = name_or_path

In [11]:
class MultitaskBERT(nn.Module):
    '''
    This module should use BERT for 3 tasks:

    - Sentiment classification (predict_sentiment)
    - Paraphrase detection (predict_paraphrase)
    - Semantic Textual Similarity (predict_similarity)
    '''
    def __init__(self, config):
        super(MultitaskBERT, self).__init__()
        # You will want to add layers here to perform the downstream tasks.
        # Pretrain mode does not require updating bert paramters.
        #self.num_labels = config.num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')#, local_files_only=config.local_files_only)
        for param in self.bert.parameters():
            if config.option == 'pretrain':
                param.requires_grad = False
            elif config.option == 'finetune':
                param.requires_grad = True
        ## sst - sentiment, para, sts - semantic text similarity
        self.num_labels_sst = 5    
        self.drop = torch.nn.Dropout(p=0.3)
        self.sst_classifier = torch.nn.Linear(self.bert.config.hidden_size, self.num_labels_sst)
        self.para_classifier = torch.nn.Linear(self.bert.config.hidden_size, 1)
        self.sts_classifier = torch.nn.Linear(self.bert.config.hidden_size, 1)



    def forward(self, input_ids, attention_mask):
        'Takes a batch of sentences and produces embeddings for them.'
        # The final BERT embedding is the hidden state of [CLS] token (the first token)
        # Here, you can start by just returning the embeddings straight from BERT.
        # When thinking of improvements, you can later try modifying this
        # (e.g., by adding other layers).
        bert_out = self.bert(input_ids, attention_mask) 
        dropped = self.drop(bert_out['pooler_output'])
        out = self.linear(dropped)
        sentence_embeddings = out.last_hidden_state[:, 0, :]
        return sentence_embeddings

In [12]:
from bert import BertModel
config = BertConfig(option='finetune')
bert_mod = bert.BertModel(config)

model = MultitaskBERT(config)

In [12]:
tk_type_embedding = nn.Embedding(30522, 768)
random_integers = torch.randint(low=0, high=100, size=(100, 100), dtype=torch.int32)



    # Get word embedding from self.word_embedding into input_embeds.

tk_type_ids = torch.zeros(random_integers.size(), dtype=torch.long)
tk_type_embeds = tk_type_embedding(tk_type_ids)

NameError: name 'nn' is not defined

In [21]:
tk_type_embeds.size()

torch.Size([100, 100, 768])

In [43]:
class BertModel(BertPreTrainedModel):
  """
  the bert model returns the final embeddings for each token in a sentence
  it consists
  1. embedding (used in self.embed)
  2. a stack of n bert layers (used in self.encode)
  3. a linear transformation layer for [CLS] token (used in self.forward, as given)
  """
  def __init__(self, config):
    super().__init__(config)
    self.config = config

    # embedding
    self.word_embedding = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
    self.pos_embedding = nn.Embedding(config.max_position_embeddings, config.hidden_size)
    self.tk_type_embedding = nn.Embedding(config.type_vocab_size, config.hidden_size)
    self.embed_layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
    self.embed_dropout = nn.Dropout(config.hidden_dropout_prob)
    # position_ids (1, len position emb) is a constant, register to buffer
    position_ids = torch.arange(config.max_position_embeddings).unsqueeze(0)
    self.register_buffer('position_ids', position_ids)

    # bert encoder
    self.bert_layers = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])

    # for [CLS] token
    self.pooler_dense = nn.Linear(config.hidden_size, config.hidden_size)
    self.pooler_af = nn.Tanh()

    self.init_weights()

  def embed(self, input_ids):
    input_shape = input_ids.size()
    seq_length = input_shape[1]

    # Get word embedding from self.word_embedding into input_embeds.
    inputs_embeds = self.word_embedding(input_ids)

    # Get position index and position embedding from self.pos_embedding into pos_embeds.
    pos_ids = self.position_ids[:, :seq_length] #subsets a list of positions 0:512 to 0:seq_length. 

    pos_embeds = self.pos_embedding(pos_ids)


    # Get token type ids, since we are not consider token type, just a placeholder.
    #tk_type_ids = torch.zeros(input_shape, dtype=torch.long, device=input_ids.device)
    tk_type_ids = create_attention_mask(input_ids)
    tk_type_embeds = self.tk_type_embedding(tk_type_ids)

    # Add three embeddings together; then apply embed_layer_norm and dropout and return.
    hidden_states = inputs_embeds+pos_embeds+tk_type_embeds
    hidden_states = self.embed_layer_norm(hidden_states)
    hidden_states = self.embed_dropout(hidden_states)

    return hidden_states, tk_type_ids


  def encode(self, hidden_states, attention_mask):
    """
    hidden_states: the output from the embedding layer [batch_size, seq_len, hidden_size]
    attention_mask: [batch_size, seq_len]
    """
    # get the extended attention mask for self attention
    # returns extended_attention_mask of [batch_size, 1, 1, seq_len]
    # non-padding tokens with 0 and padding tokens with a large negative number 
    extended_attention_mask: torch.Tensor = get_extended_attention_mask(attention_mask, self.dtype)

    # pass the hidden states through the encoder layers
    for i, layer_module in enumerate(self.bert_layers):
      # feed the encoding from the last bert_layer to the next
      hidden_states = layer_module(hidden_states, extended_attention_mask)

    return hidden_states

  def forward(self, input_ids, attention_mask):
    """
    input_ids: [batch_size, seq_len], seq_len is the max length of the batch
    attention_mask: same size as input_ids, 1 represents non-padding tokens, 0 represents padding tokens
    """
    # get the embedding for each input token
    embedding_output = self.embed(input_ids=input_ids)

    # feed to a transformer (a stack of BertLayers)
    sequence_output = self.encode(embedding_output, attention_mask=attention_mask)

    # get cls token hidden state
    first_tk = sequence_output[:, 0]
    first_tk = self.pooler_dense(first_tk)
    first_tk = self.pooler_af(first_tk)

    return {'last_hidden_state': sequence_output, 'pooler_output': first_tk}

In [11]:
config = BertConfig()
BertModel.embed()

torch.Size([1, 512])

In [8]:
class tempArgs():
    def __init__(self,):
        self.local_files_only=True
tempargs = tempArgs()

In [56]:
from datasets import SentencePairDataset
from datasets import load_multitask_data, SentenceClassificationDataset, SentencePairDataset
from config import PretrainedConfig
import torch
from base_bert import BertPreTrainedModel
max_length = 600
sentiment_data, num_labels, paraphrase_data, similarity_data = load_multitask_data(sentiment_filename='data/ids-sst-train.csv',paraphrase_filename='data/quora-train.csv',similarity_filename='data/sts-train.csv')
para_train_data = SentencePairDataset(paraphrase_data,tempargs,isRegression=False)
batched_para_data = para_train_data.collate_fn(para_train_data.dataset)
concatenation = concatenate_with_padding_and_mask(batched_para_data['token_ids_1'],batched_para_data['token_ids_2'],batched_para_data['attention_mask_1'],batched_para_data['attention_mask_2'])
concatenation=concatenation.to(torch.long)


Loaded 8544 train examples from data/ids-sst-train.csv
Loaded 141498 train examples from data/quora-train.csv
Loaded 6040 train examples from data/sts-train.csv


In [23]:
BertModel.embed(input_ids=concatenation)

TypeError: embed() missing 1 required positional argument: 'self'

In [1]:
def find_first_occurrence(tensor):
    # Find the index of the first occurrence of value 102 (SEP token) for each row
    first_occurrence_indices = []
    for index, row in enumerate(tensor):
        index = torch.nonzero(row == 102, as_tuple=False)
        if len(index) > 0:
            first_occurrence_indices.append(index[0, 0].item())
        else:
            print(index)
            first_occurrence_indices.append(None)
    return first_occurrence_indices

In [2]:
def concatenate_with_padding_and_mask(tensor1, tensor2,mask1,mask2):
    concatenated_tensors = []

    
    

    for i in range(len(tensor1)):
        non_zero_indices1 = torch.nonzero(mask1[i], as_tuple=False).squeeze()
        non_zero_indices2 = torch.nonzero(mask2[i], as_tuple=False).squeeze()
        
        # Get the maximum non-zero index for each tensor separately
        max_index1 = torch.max(non_zero_indices1).item() + 1
        max_index2 = torch.max(non_zero_indices2).item() + 1

        # Concatenate the tensors and update attention masks
        # the second tensor concatenatesfrom position 1 because the separator 
        concatenated_tensor = torch.cat((tensor1[i][:max_index1], tensor2[i][1:max_index2]), dim=0)

        # Pad the concatenated tensor and mask to the maximum length
        padding_length = max_length - len(concatenated_tensor)
        padding = torch.zeros(padding_length)

        concatenated_tensor = torch.cat((concatenated_tensor, padding), dim=0)

        concatenated_tensors.append(concatenated_tensor)

    return torch.stack(concatenated_tensors)

In [72]:
config = BertConfig()
bert_model = BertModel(config)

# Prepare example input data (input_ids)
input_ids = torch.tensor([[1, 2, 3, 4, 102, 6, 102]])  # Replace with your input token IDs

# Call the embed method
embeddings, tk_type_ids = bert_model.embed(input_ids=input_ids)

tensor([[0, 0, 0, 0, 0, 1, 1]])